このチュートリアルは自然言語処理で深層学習を行うコードを，1から学ぶための資料です．自然言語処理に於ける深層学習の中心的な手法である，埋め込みベクトルについての説明から始まり，埋め込みベクトルの学習（word2vec），再帰ニューラルネット，系列変換モデル（seq2seq）について解説します．

In [1]:
import collections
import numpy
import chainer
import chainer.functions as F
import chainer.links as L

# 前処理
自然言語処理が扱うデータは文字列です．Pythonでは`str`型で渡されるのが一般的です．深層学習に限りませんが，自然言語処理では単語の単位に分けて処理を行うのが一般的です．ここでは，`str.split`メソッドを呼ぶことで単語に分けましょう．そして，各単語に対してユニークな整数値のIDを割り当てあてます．ここでは，`collections.defaultdict`を使うことで，単語IDを割り当てましょう．

In [2]:
sentence = 'hello chainer world'
word_ids = collections.defaultdict(lambda: len(word_ids))
words = sentence.split()
print(words)
x = numpy.array([word_ids[w] for w in words], dtype=numpy.int32)
print(x)

['hello', 'chainer', 'world']
[0 1 2]


余談ですが，どのような単位で処理を行うのが最適化は自明な問題ではありません．単語ではなく，文字単位で処理を行ったほうがよいという研究もあります．この場合，`str.split`メソッドで分割するのではなくて，1文字ずつ分割することで実現できます．

# 埋め込みベクトル
自然言語処理が扱う対象のデータは，単語や文字といった離散的な記号です．こうした離散的な対象を深層学習で扱うのに使われるのが，埋め込みベクトルです．
離散的な対象を扱うのは非常に厄介です．単語の異なり数は，数万〜数百万にも及びますし，非常に出現頻度の少ない単語も存在します．一方，単語の表層が似ている，つまり文字列として似ていることが必ずしも意味的に似ているとは限りません．「リンゴ」と「リンク」は1文字違いですが意味的なつながりはまったくありませんが，「リンゴ」と「ミカン」には1文字も共通点がりませんが意味の類似性があります．このように，離散的な記号を扱うときには，記号の異なり数が非常に大きいことと，記号間の類似度をうまく扱う必要があります．そこで，各記号（ここでは単語）ごとに1つの多次元ベクトルを割り当てることにしましょう．多次元ベクトル同士は，例えば内積などの演算によってベクトル間の距離（類似度）を定義することができます．理想的には意味の似ている単語に，距離の近いベクトルが割り当てられることが望まれます．このようなベクトルを埋め込みベクトルと呼びます．
L.EmbedIDは，各単語ごとに割り当てられるベクトルをパラメータとする関数です．パラメータ付きの関数は，ChainerではLinkと呼ばれます．EmbedIDは，単語の異なり数 x 埋め込みベクトルの次元数のサイズの行列をパラメータとして持ちます．このLinkのインスタンスを作り，整数列を渡すことで，埋め込みベクトル列を得ることができます．ここでは，埋め込みベクトルの次元は5としました．

In [23]:
n_unit = 5
embed = L.EmbedID(len(word_ids), n_unit)
e = embed(x)
print(e.data)

[[-0.99803847 -0.78388876 -0.32097885  0.40594256 -0.00573746]
 [ 1.63370097  0.75120306 -0.24381094  0.18228573  2.32663512]
 [-0.41459712  0.41527957  0.98071295 -0.31959069 -1.10089731]]


各埋め込みベクトルは乱数で初期化されているため，このベクトルにはまだ意味はありません．学習データを使って目的関数を設計し，それを最小化するように埋め込みベクトルのパラメータを得ることを考えましょう．例えば，（工学的な意味はありませんが）単純に学習データで出現する埋め込みベクトルの平均ベクトルのL1ノルムを最小化してみます．
学習には，Optimizerと呼ばれるオブエジェクトを作り，setupメソッドで学習対象のパラメータを持つLinkを指定します．損失を計算したら，損失を持つVariableのbackwardメソッドを呼ぶと，各パラメータの勾配情報が誤差逆伝播によって更新されます．その後，optimizerのupdateメソッドを呼ぶとパラメータが更新されます．これを何度も繰り返します．

In [25]:
optimizer = chainer.optimizers.SGD()
optimizer.setup(embed)

for i in range(100):
    e = embed(x)
    mean = F.average(e, axis=0)
    loss = F.sum(F.absolute(mean))
    if i % 10 == 0:
        # show log
        print('%d: %f' % (i, loss.data))

    embed.cleargrads()
    loss.backward()
    optimizer.update()

0: 0.078792
10: 0.045458
20: 0.012125
30: 0.005458
40: 0.005458
50: 0.005458
60: 0.005458
70: 0.005458
80: 0.005458
90: 0.005458


段々と損失の値が下がっているのがわかると思います．ChainerにはTrainerと呼ばれる機能があり，このforループ（学習ループと呼ばれます）を抽象化することもできます．ここではTrainerは使わずに直接ループをかいています．